Лаба 4b. Обучение и инференс с использованием SparkML, sci-kit learn.
Дедлайн
⏰ 21 июня 2021 года, 23:59.

------------------------
Данная работа может быть сделана 2 способами:

Простой способ. Реализация работы на SparkML с базовыми трансформерами и эстиматорами.

Усложненный способ. Реализация работы на Spark ML с пользовательскими трансформерами и эстиматорами и подключением кода обучения модели на питоне. Дело тут вот в чем. SparkML - довольно ограниченная библиотека, по крайней мере с точки зрения дата-сайентиста. В ней нет многих алгоритмов, к которым привыкли ДС. Однако спарк обладает несомненным преимуществом в параллельной обработке большого количества данных. А так же разработчики спарка дали возможность расширять библиотеку собственными классами. Это позволит объединить оба мира, взяв от каждого лучшее.

Задача с высоты птичьего полета.
Обучите модель <b>LogisticRegression</b> на датасете weblogs, используя SparkML и Pipeline.

Разработайте приложение на Spark для инференса тестового датасета из Kafka, используя Spark Structured Streaming.

Если Вы решили реализовывать собственные трансформеры и эстиматоры и глубже разобраться как работает SparkML под капотом, то:

Добавьте в этот пайплайн собственноручно написанный класс-трансформер для преобразования url в domain.

Напишите собственный класс-эстиматор. В этом эстиматоре в методе fit() вы будете вызывать программу для обучения модели на питоне с использованием sklearn, в которой подаются данные для обучения, а обученную модель вы сохраняете в переменной класса. В методе transform() вы тоже будете вызывать питоновский скрипт, применяющий модель sklearn для предсказаний.

Обученный пайплайн сохраняется на диск обычным образом. Далее вы применяете тот же самый код для инференса.

Такой вариант может показаться спорным, ведь питоновская программа для обучение должна запускаться один раз на драйвере, и будет ограничена по памяти, то есть ей придется давать датасет небольшого размера. Но на практике это не так страшно - сэмплируют небольшой датасет из большого и это не приводит к существенной потере качества модели.

In [1]:
import scala.math.max
import org.apache.spark.rdd.RDD
import scala.io.Source
import java.io._
import org.apache.spark.sql.functions._
import org.apache.spark.mllib.linalg.distributed._
import org.apache.spark.ml.linalg.{SparseVector, Vector}
import org.apache.spark.sql.types._
import org.apache.spark.ml.feature._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

In [2]:
import org.apache.spark.ml.classification.LogisticRegression
import org.apache.spark.ml.feature.{CountVectorizer, StringIndexer, IndexToString}
import org.apache.spark.ml.{Pipeline, PipelineModel}

In [28]:
import org.apache.spark.sql.Row

In [3]:
val spark = SparkSession
   .builder()
   .appName("makoles lab4b")
   .config("spark.executor.instances", "16")
   .getOrCreate()

spark = org.apache.spark.sql.SparkSession@31ae15ab


org.apache.spark.sql.SparkSession@31ae15ab

II. Описание данных
Обучающая выборка, с которой вы будете работать, выглядит следующим образом (датасет weblogs (путь на hdfs: /labs/slaba04b/laba04b.json), в котором добавлены метки класса для обучения).

In [4]:
val data_set = spark.read.json("/labs/slaba04b/laba04b.json")

data_set = [gender_age: string, uid: string ... 1 more field]


[gender_age: string, uid: string ... 1 more field]

In [5]:
data_set.printSchema

root
 |-- gender_age: string (nullable = true)
 |-- uid: string (nullable = true)
 |-- visits: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- timestamp: long (nullable = true)
 |    |    |-- url: string (nullable = true)



In [6]:
data_set.limit(3).show

+----------+--------------------+--------------------+
|gender_age|                 uid|              visits|
+----------+--------------------+--------------------+
|   F:45-54|0dce7423-1f2e-4db...|[[1419604951975, ...|
|   M:25-34|0dcf8ee3-6b33-457...|[[1419869055980, ...|
|   M:45-54|0dd1aa68-b788-414...|[[1427117036000, ...|
+----------+--------------------+--------------------+



Поле uid принимает значения уникального ID пользователя (cookies): d50192e5-c44e-4ae8-ae7a-7cfe67c8b777.

In [7]:
val data_set_pars_visit=data_set
    .withColumn("visit",explode('visits))
    .drop("visits")
    .withColumn("url",col("visit.url"))
    .select('uid,'gender_age, 'url)

data_set_pars_visit = [uid: string, gender_age: string ... 1 more field]


[uid: string, gender_age: string ... 1 more field]

Регулярное выражение для обрезки домена из url адреса

In [8]:
val new_re = """^http(s)?(%3A|:)(//|%2F%2F)(www\.)?(([\%a-zA-Zа-яА-Я0-9-]+\.)+[\%a-zA-Zа-яА-Я0-9-]+)\.?/.*$"""

new_re = ^http(s)?(%3A|:)(//|%2F%2F)(www\.)?(([\%a-zA-Zа-яА-Я0-9-]+\.)+[\%a-zA-Zа-яА-Я0-9-]+)\.?/.*$


^http(s)?(%3A|:)(//|%2F%2F)(www\.)?(([\%a-zA-Zа-яА-Я0-9-]+\.)+[\%a-zA-Zа-яА-Я0-9-]+)\.?/.*$

In [9]:
val data_set_domain=data_set_pars_visit
    .withColumn("domain", regexp_extract(concat(regexp_replace('url, "%2F", "/"), lit("/")), new_re, 5))
    .filter('domain !== "")
    .select('uid,'gender_age, 'domain)

data_set_domain = [uid: string, gender_age: string ... 1 more field]


[uid: string, gender_age: string ... 1 more field]

In [10]:
val data_set_group=data_set_domain
    .groupBy(data_set_domain("gender_age"), data_set_domain("uid"))
    .agg(collect_list(data_set_domain("domain")).alias("domains"))

data_set_group = [gender_age: string, uid: string ... 1 more field]


[gender_age: string, uid: string ... 1 more field]

In [11]:
val cv = new CountVectorizer()
      .setInputCol("domains")
      .setOutputCol("features")

val indexer = new StringIndexer()
      .setInputCol("gender_age")
      .setOutputCol("label")

val lr = new LogisticRegression()
      .setMaxIter(32)
      .setRegParam(0.000001)
      .setElasticNetParam(0.2)

val labels = indexer.fit(data_set_group).labels

val converter = new IndexToString()
  .setInputCol("prediction")
  .setLabels(labels)
  .setOutputCol("res")

val pipeline = new Pipeline()
  .setStages(Array(cv, indexer, lr, converter))

cv = cntVec_f95528cb218e
indexer = strIdx_c3a38b5429e9
lr = logreg_360325ee4624
labels = Array(M:25-34, F:25-34, M:35-44, F:35-44, F:18-24, F:45-54, M:45-54, M:18-24, F:>=55, M:>=55)
converter = idxToStr_cad51441b39f
pipeline = pipeline_2585360ad4ef


pipeline_2585360ad4ef

In [12]:
val model = pipeline.fit(data_set_group)

model = pipeline_2585360ad4ef


pipeline_2585360ad4ef

In [13]:
val train_predictions = model.transform(data_set_group)

train_predictions = [gender_age: string, uid: string ... 7 more fields]


[gender_age: string, uid: string ... 7 more fields]

In [14]:
train_predictions.filter('uid === "d50192e5-c44e-4ae8-ae7a-7cfe67c8b777").show(true)

+----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+-------+
|gender_age|                 uid|             domains|            features|label|       rawPrediction|         probability|prediction|    res|
+----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+-------+
|   F:18-24|d50192e5-c44e-4ae...|[zebra-zoya.ru, n...|(107846,[103,5325...|  4.0|[-3.5707225774695...|[1.28722821284297...|       4.0|F:18-24|
+----------+--------------------+--------------------+--------------------+-----+--------------------+--------------------+----------+-------+



In [15]:
train_predictions.withColumn("res_correct", expr("case when res = gender_age then 1 else 0 end"))
.groupBy('gender_age, 'res_correct)
.count()
.orderBy('gender_age, 'res_correct).show(100)

+----------+-----------+-----+
|gender_age|res_correct|count|
+----------+-----------+-----+
|   F:18-24|          0|  633|
|   F:18-24|          1| 2252|
|   F:25-34|          0|  728|
|   F:25-34|          1| 6061|
|   F:35-44|          0|  695|
|   F:35-44|          1| 3576|
|   F:45-54|          0|  534|
|   F:45-54|          1| 2063|
|    F:>=55|          0|  192|
|    F:>=55|          1|  703|
|   M:18-24|          0|  391|
|   M:18-24|          1| 1621|
|   M:25-34|          0|  773|
|   M:25-34|          1| 7893|
|   M:35-44|          0|  712|
|   M:35-44|          1| 4373|
|   M:45-54|          0|  342|
|   M:45-54|          1| 1804|
|    M:>=55|          0|  122|
|    M:>=55|          1|  660|
+----------+-----------+-----+



In [16]:
model.write.overwrite().save("model_lab04b.ml")

spark-master-1.newprolab.com:6667
spark-node-1.newprolab.com:2181

In [17]:
val topic_in = "maria_kolesnikova"
val topic_out = "maria_kolesnikova_lab04b_out"
val kafka_bootstrap = "spark-master-1:6667"

topic_in = maria_kolesnikova
topic_out = maria_kolesnikova_lab04b_out
kafka_bootstrap = spark-master-1:6667


spark-master-1:6667

Задали схему для потока из Кафки:

In [18]:
val schema = StructType(Seq(
  StructField("uid", StringType, true),
  StructField("visits", ArrayType(StructType(Seq(StructField("timestamp",LongType,true), StructField("url", StringType, true))), true), true)
))

schema = StructType(StructField(uid,StringType,true), StructField(visits,ArrayType(StructType(StructField(timestamp,LongType,true), StructField(url,StringType,true)),true),true))


StructType(StructField(uid,StringType,true), StructField(visits,ArrayType(StructType(StructField(timestamp,LongType,true), StructField(url,StringType,true)),true),true))

In [19]:
import java.net.URL
import java.net.URI
import util.Try
import java.net.MalformedURLException
def url2domain (url: String) : String = {
    var res_domain = ""
    try {
        res_domain = new URL(url).getHost
    } catch {
            case e: MalformedURLException => res_domain = new URI(url.replace(" ", "")).getHost
    }
    res_domain
    }

url2domain: (url: String)String


In [20]:
url2domain("http://www.toysrus.com/category/index.jsp?categoryid=57525526&sr=1&origkw=monster high")

www.toysrus.com

In [29]:
def converturls = udf { (url_arr: Seq[Row]) => 
    url_arr.map(x => url2domain(x.getAs[String]("url")) ).distinct
    }

converturls: org.apache.spark.sql.expressions.UserDefinedFunction


In [30]:
val df_in = spark
    .readStream 
    .format("kafka") 
    .option("checkpointLocation", "/tmp/checkpoint-aokorogo-read") 
    .option("kafka.bootstrap.servers", kafka_bootstrap) 
    .option("subscribe", topic_in) 
    .option("startingOffsets", "latest") 
    .option("failOnDataLoss", false)
    .load()
    .selectExpr("CAST(value AS STRING)")
    .withColumn("jsonData", from_json(col("value"), schema))
    .select("jsonData.*")
    .select('uid, converturls(col("visits")).alias("domains"))


df_in = [uid: string, domains: array<string>]


[uid: string, domains: array<string>]

In [31]:
df_in.printSchema

root
 |-- uid: string (nullable = true)
 |-- domains: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [32]:
val results = model.transform(df_in)

results = [uid: string, domains: array<string> ... 5 more fields]


[uid: string, domains: array<string> ... 5 more fields]

In [33]:
results.printSchema

root
 |-- uid: string (nullable = true)
 |-- domains: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)
 |-- res: string (nullable = true)



In [34]:
val df_out = results
    .withColumnRenamed("res", "gender_age")
    .select("uid", "gender_age")

df_out = [uid: string, gender_age: string]


[uid: string, gender_age: string]

In [35]:
val query = df_out
    .select(to_json(struct("uid", "gender_age")).alias("value"))
    .writeStream
    .outputMode("append")
    .format("kafka")
    .option("checkpointLocation", "/tmp/checkpoint-write")
    .option("kafka.bootstrap.servers", kafka_bootstrap )
    .option("topic", topic_out)
    .start()

query = org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@61039bdf


org.apache.spark.sql.execution.streaming.StreamingQueryWrapper@61039bdf

In [ ]:
query.awaitTermination()
query.stop()

In [ ]:
spark.stop()